# 04.VAE

참고 : AE를 언제 쓰나?
- AE로 차원 축소를 해보고 그 결과를 classic한 Dim. reduction방법(t-SNE... etc.)으로 한번 더 해본다.
- 차원 축소가 잘 되었다면 manifold상에서 거리가 가깝다는 것이다. 고전적인 차원 축소 방식은 neighborhood based 에서 잘 동작하므로 manifold상에서 가까이 있다면 classic한 방법으로 차원축소를 했을때도 잘 동작할 것이기 때문이다.

### Generative Model

VAE & AE
- VAE : Generative model learning, 생성 모델, 네트워크 뒷단의 학습이 목적이다.
- AE : Manifold learning, 네트워크 앞단의 학습이 목적이다.
- 나중에 설명하겠지만 VAE와 AE는 네트워크 모양만 비슷할 뿐 수식적으로 아무 관련이 없다.

![img25](./img/img25.png)
- Generator 모델은 트레이닝 DB에서 우리가 원하는 데이터 x가 나올 확률을 구하는데 모든 트레이닝 DB에 대하여 그 확률을 최대화하는 확률 분포를 구하는 것이 목적이다.
- generator 모델에서 생성되는 데이터는 랜덤하게 나오는 거보다 컨트롤 할 수 있는것이 좋다.
    - 예를 들어 첫번째 element를 바꾸면 성별이 바뀐다던지
- z는 리모컨같은 역할을 하는 것. prior확률분포 에서 샘플링돼서 나오는 값.$z\sim p(z)$
- 네트워크 출력값 자체는 deterministic한 값. 확률분포의 parameter를 추정한다고 생각한다.$g_{\theta}(\cdot)$
- 우리가 알고 싶은것은 모든 트레이닝 DB에 대한 확률분포 $ p(x) $ 는 네트워크 추정값인 $ p(x|g_{\theta}(z))$와 미리 정해놓은 prior 확률분포 $ p(z)$를 이용해서 구한다.$$ \int p(x|g_{\theta}(z))p(z) dz = p(x)$$ 
- z를 어떻게 Generator에 넣는냐에 따라서 출력값이 달라지므로 $p(z)$우리가 다루기 쉬운 확률분포를 사용한다. normal or uniform distribution
- 그런데 $p(z)$를 단순한 분포를 사용해도 상관이 없나??
    - z는 결국 latent space(manifold) 상에 있는 값을 취하는것. 복잡할 것 같은 공간인데 단순한 확률분포를 사용하는 것이 그 공간을 잘 설명하는 것이 맞는가??
    - 어차피 Deep NN에서 앞쪽에 레이어 1~2개가 복잡한 latent space를 익히므로 prior distribution은 단순한 걸 써도 괜찮다.
    - ref)https://arxiv.org/pdf/1606.05908.pdf figure2
    - 단순한 분포에서 z를 뽑아도 이후에 약간의 테크닉만으로도 완전히 다른 분포를 만들어낼 수 있다. 

- $p(x)$를 구하기 위한 값을 모두 알고 있는 값인데 왜 바로 MLE하지 않는가? 샘플별로 구해서 summation하면 구할 수 있지 않은가?
-  안된다!! VAE로 해야한다.
![img26](./img/img26.png)
- 위 그림에서 (a)는 원 샘플, (b)는 일부분을 지운것, (c)는 (a)에서 화소값을 왼쪽으로 1씩 쉬프트한것이다. 의미적으로 봤을때 (a)와 (c)가 더 가까워 보인다.
- 하지만 MSE를 구해보면 (a)와 (b)의 MSE가 더 작게 나온다. 모델을 MSE가 작은쪽으로 학습하므로 이렇게 되면 학습된 모델에서 샘플링했을때 (c)보다 (b)가 나올 확률이 높아진다.(우리가 원하는 건 의미적으로 더 가까운 (c)인데)
- 즉, MSE가 작다고 해서 의미적으로 가까운것이 아니라는것. **Sampling 함수를 잘 정해야한다.**
    - 어떤게 이상적인 Sampling함수인지 모르므로 추정해야 한다. Variational Inference를 통해서!

### Variational Inference

이상적인 Sampling함수는 결국 $x$와 유사한 샘플이 나올수 있는 확률분포 $p(z|x)$ (True Posterior) 인데 무엇인지 알 수 없으므로 우리가 알고있는 확률분포 $ q_{\phi}(z|x)$의 parameter(가우시안이라면 평균, 표준편차)를 바꿔가면서 $p(z|x)$와 유사하게 만들어간다.(Approximation class)

그렇다면 지금까지 나온 $p(x)$, $p(z)$, $p(z|x)$, $q_{\phi}(z|x)$ 간의 관계(**Evidence Lower Bound**)를 유도하는 방법이 2가지가 있는데 Derivation 2 가 더 직관적이고 다른 자료에서도 많이 볼 수 있는 방법이다. 

![img27](./img/img27.png)
- Jensen's Inequality를 이용해서 유도하는 방식이다. 
- Jensen's Inequality 개념 참고 : https://www.youtube.com/watch?v=F-_BmRx0QyU - 꼼수수학
- 우리의 target distribution인 $p(x)$ 보다 항상 작거나 같은 값이 나오기 때문에 lower bound라고 부른다. 

![img28](./img/img28.png)
- 조건부 확률의 정의를 이용해서 정리하면 term이 2개가 나오는데 ELBO는 Derivation 1에서 구한 ELBO와 같은 것이다.
- KL은 True Posterior(이상적인 Sampling 확률분포)와 Approximation(이상적인 Sampling함수를 추정하기 위해 도입한 임의의 확률분포)의 거리를 나타낸다. 즉 KL을 minimize해야한다.
- 그런데 위 식에서 KL을 minimize하는 것은 **ELBO를 maximize**하는 것과 같다. $$ q_{\phi^{*}}(z|x) = \underset{\phi}{\mathrm{argmax}}ELBO(\phi)$$
- 여기서 ELBO를 전개하면 KL이 또 나오는데 앞에 나온 KL과 다른 것이다. (인자가 다름)
- ![img29](./img/img29.png)


결과적으로 우리는 2개의 Optimization Problem을 한꺼번에 푸는 것이다.
1. Optimization Problem 1 on $\phi$ : ELBO를 maximize하는 과정. 즉, 이상적인 Sampling함수를 찾는 과정.$$ log(p(x)) \ge \mathbb{E}_{q_{\phi}(z|x)} {\left[ log(p(x|z)) \right]} - {KL(q_{\phi}(z|x) \parallel p(z) )} = ELBO(\phi) $$
1. Optimization Problem 2 on $\theta$ : Generator에서 $x$가 나올 확률을 maximize하는 과정(Maximum Likelihood). 네트워크 학습과정.$$ -\sum_{i} log(p(x_{i})) \le -\sum_{i} {\left\{ \mathbb{E}_{q_{\phi}(z|x_{i})} {\left[ log(p(x_{i}|{\color{Red}{g_{\theta}(z)}})) \right]} - {KL(q_{\phi}(z|x_{i}) \parallel p(z) )} \right\}}$$

그런데 Maximum Likelihood하는것은 ELBO식에 이미 들어가있다. 따라서 최종 Optimization Problem은 $$ \underset{\phi,\theta}{\mathrm{argmin}} \sum_{i} - {\mathbb{E}_{q_{\phi}(z|x_{i})} {\left[ log(p(x_{i}|{g_{\theta}(z)})) \right]} + {KL(q_{\phi}(z|x_{i}) \parallel p(z) )} } $$
- $\phi$에 대해서 ELBO를 maximize하면 Variational Inference를 푸는것이고 $\theta$에 대해서 maximize하면 Maximum Likelihood를 하는것이다.  

![img30](./img/img30.png)
- 결국 Decoder(generator)를 학습시키기 위해서 이상적인 Sampling함수가 필요한데 그 Sampling함수를 찾기 위한 역할을 하는 Encoder가 앞단에 붙은 형태가 VAE의 큰 그림이다.


### Loss Function

![img31](./img/img31.png)
- Reconstruction Error : Maximum Likelihood관점에서 Optimization해야되는 것. $x$를 넣으면 $x$가 잘 나오니?(Sampling이 들어간것 빼고는 AE의 term과 같은 개념)
- Regularization : 같은 Reconstruction Error를 가진다면 여러개의 $q_{\phi}(z|x)$ 중에 prior($p(z)$)와 같은 모양인 것을 선택하여라.

코딩을 위해서 이것을 식으로 만들어보자. 계산을 위해 2가지 가정을 한다.
1. $q_{\phi}(z|x_{i}) \sim N(\mu_{i}, \sigma_{i}^{2}I)$ (Approximation은 Gaussian distribution)
1. $p(z) \sim N(0,I)$ (prior는 Normal distribution) 


##### KL Divergence

![img32](./img/img32.png)
- 2개의 Gaussian 분포 사이의 KL divergence는 이미 수학적으로 알려져있다. (Normal distribution도 gaussian의 종류이므로) 구해보면 $$\frac{1}{2}\sum_{j=1}^{J}(\mu^{2}_{i,j}+\sigma^{2}_{i,j}-\ln(\sigma^{2}_{i,j})-1)$$

In [ ]:
KL_divergence = 0.5 * tf.reduce_sum(tf.square(mu) + tf.square(sigma) - tf.log(1e-8 + tf.square(sigma)) - 1, 1)

수식 그대로 구현한 것. 다른 VAE코드들에서는 상수들이 생략된 경우가 있다. Optimization Problem이므로 상수 term은 영향을 미치지 않으므로.

##### Sampling

![img33](./img/img33.png)
- Monte Carlo technique 을 이용하는데 이것은 $L$개를 Sampling을 하는 것이다. Sampling을 하면 random node가 만들어지는데 Backpropagation할 때 중간값이 계속 바뀌게 되므로 chain rule을 쓸 수 없어서 정상적으로 Backpropa가 작동할 수 없다. 이 문제를 해결하기 위해 **Reparameterization Trick**을 쓴다.
- ![img34](./img/img34.png)
    - Original form에서 sampling을 한 것과 reparameterised form에서 sampling한 확률변수는 같은 확률적 특성을 가진다.(수학적으로 알려져 있는 사실)
    - 그런데 random node가 chain rule을 쓸 때 영향을 미치지 않으므로 Backpropa를 할 수 있다.

In [ ]:
# sampling by re-parameterization technique
z = mu + sigma * tf.random_normal(tf.shape(mu), 0, 1, dtype=tf.float32)

- 그런데 주로 여러개를 Sampling을 잘 안하고 한 개만 Sampling한다. 
- ![img35](./img/img35.png)
- ![img36](./img/img36.png)
- 보통 이미지 분야에서는 conditional probability를 Bernoulli로 가정하고 likelihood를 구한다. 그러면 결국 이전에 봤던것처럼 Cross Entropy값이 된다. 만약에 Gaussian으로 가정하고 구하면 MSE값이 된다.

#### Summary

![img37](./img/img37.png)
- 논문에 보면 Decoder부분의 확률 분포는 다르게 쓰인 형태가 있는데 Encoder부분은 바뀌지 않는다. 그 이유는 Gaussian분포 이외의 확률 분포에서 KL divergence를 구하는 것이 힘들기 때문이다. 그 한계를 극복한 것이 Adversarial AE이다.
- code : https://github.com/hwalsuklee/tensorflow-mnist-VAE/blob/master/vae.py

#### Result
- Reproduce : Reconstruction 관점에서 z의 dimension이 클수록 복원이 잘되는 것을 볼 수 있다.
- Denoising : noise를 추가해도 복원이 잘 된다.

#### AE vs VAE

![img38](./img/img38.png)
- 데이터 생성 관점에서 다루고자 하는 manifold의 위치가 안정적인 것이 좋다.
- AE의 경우 manifold가 맵핑되는 위치가 계속해서 바뀌는 반면 VAE는 Normal distribution을 따르는 것을 볼 수 있다.
    - 이것은 KL term의 유무로 나타나는 것이다. VAE의 경우 이상적인 Sampling함수가 결국에는 Prior와 똑같게 만들어지므로(KL의 역할) 그 Prior를 잘 해석하도록 학습된 Generator는 안정적으로 manifold를 찾는다.

![img39](./img/img39.png)
- Prior를 Normal distribution으로 줬기 때문에 위와 같은 manifold로 맵핑된다.
- 생성된 이미지를 보면 size, rotation 같은 feature들이 저절로 찾아진 것을 볼 수 있다.

### CVAE

### AAE